As required in the question, we have to get a sorted range.  
Since it has to be sorted, we can use Tree, or List with binary search.  
Here I use Deque with binary search, because:  
- query index for impact ranges with binary search, which is O(logN)  
- deque can have O(1) to appendleft or append  
- rotate with O(k) k is index  

After find out index, compare the ranges and new range to add or remove.  

so:  
- time complexity max would be O(N), Normally would be O(logN)
- Space complexity O(N)  
 


In [70]:
from typing import List
from collections import deque
import logging

logging.basicConfig(
    level=logging.DEBUG, 
    format='%(asctime)s - %(levelname)s - %(message)s',
)

logger = logging.getLogger()


class RangeList:
    def __init__(self):
        """
        Initialize a RangeList object with an empty deque to store ranges.
        This class frequently add new range and delete ranges
        It uses deque, in order to remove and add with O(1) time complexity and O(logn) to remove
        """
        self._range_list = deque()

    @staticmethod
    def _is_valid_range(a_range: List) -> bool:
        """
        Check if a given range is valid.

        Parameters:
        - a_range (list): The range to check.

        Returns:
        - bool: True if the range is valid, False otherwise.
        """
        return (
                type(a_range) is list and
                len(a_range) == 2 and
                type(a_range[0]) is int and
                type(a_range[1]) is int and
                a_range[0] < a_range[1]
        )

    def _find_value_index(self, value: int) -> int:
        """
        Find the index of the range that contains a given value with binary search, since range list is sorted.

        Parameters:
        - value (int): The value to find.

        Returns:
        - int: The index of the range containing the value.
        """
        low = 0
        high = len(self._range_list) - 1

        while low < high:
            mid = (low + high) // 2
            if self._range_list[mid][0] <= value <= self._range_list[mid][1]:
                return mid
            elif self._range_list[mid][1] <= value:
                low = mid + 1
            else:
                high = mid - 1

        return low


    def add(self, a_range: List) -> None:
        """
        Add a range to the RangeList.
        """
        if not self._is_valid_range(a_range):
            logger.error(f'Wrong input with value: {a_range}')
            return

        if not self._range_list:
            self._range_list.append(a_range)
            return

        left, right = a_range
        start_index = self._find_value_index(left)
        end_index = self._find_value_index(right)

        new_partial_ranges = []
        inserted = False

        self._range_list.rotate(-start_index)

        for _ in range(end_index - start_index + 1):
            start, end = self._range_list.popleft()

            if end < left:
                new_partial_ranges.append([start, end])
            elif right < start:
                if not inserted:
                    new_partial_ranges.append((left, right))
                    inserted = True
                new_partial_ranges.append([start, end])
            else:
                left = min(left, start)
                right = max(right, end)

        if not inserted:
            new_partial_ranges.append([left, right])

        [self._range_list.appendleft(i) for i in new_partial_ranges[::-1]]
        self._range_list.rotate(start_index)

    def remove(self, a_range):
        """
        Remove a range from the RangeList.

        Parameters:
        - a_range (list): The range to remove.
        """
        if not self._is_valid_range(a_range):
            logger.error(f'Wrong input with value: {a_range}')
            return

        if not self._range_list:
            return

        left, right = a_range
        start_index = self._find_value_index(left)
        end_index = self._find_value_index(right)

        self._range_list.rotate(-start_index)

        new_partial_ranges = []
        for _ in range(end_index - start_index + 1):
            start, end = self._range_list.popleft()
            if end <= left or right <= start:
                new_partial_ranges.append([start, end])
            else:
                if start < left:
                    new_partial_ranges.append([start, left])

                if end > right:
                    new_partial_ranges.append([right, end])
        [self._range_list.appendleft(i) for i in new_partial_ranges[::-1]]
        self._range_list.rotate(start_index)

    def __str__(self) -> str:
        """
        Convert the RangeList to a string representation.

        Returns:
        - str: String representation of the RangeList.
        """
        return ' '.join([f'[{start}, {end})' for start, end in self._range_list])




In [71]:
import unittest

class TestRangeList(unittest.TestCase):
    def test_add_invalid(self):
        test_range_list = RangeList()
        test_range_list.add(None)
        test_range_list.add(1)
        test_range_list.add(['a', 'b'])
        test_range_list.add([1,1])
        test_range_list.add([1,2,3])
           
        self.assertEqual(str(test_range_list), '')
        
    def test_add_multi_time(self):
        test_range_list = RangeList()
        
        test_range_list.add([-1, 0])
        test_range_list.add([-1, 0])
        test_range_list.add([100, 200])
        test_range_list.add([2, 4])
        test_range_list.add([500, 600])
        test_range_list.add([10, 50])
        
        self.assertEqual(str(test_range_list), '[-1, 0) [2, 4) [10, 50) [100, 200) [500, 600)')
        
    
    def test_add_end_less_than_start(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([-1, 0])
        self.assertEqual(str(test_range_list), '[-1, 0) [1, 4) [5, 100) [200, 300)')
    
    def test_add_already_contained(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([1, 3])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 100) [200, 300)')
        
    def test_add_merge(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([100, 201])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 300)')

        
    def test_add_merge2(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([-1, 1])
        self.assertEqual(str(test_range_list), '[-1, 4) [5, 100) [200, 300)')

        
    def test_add_merge3(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([100, 201])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 300)')
    
    def test_add_start_greater_than_max(self):
        test_range_list = RangeList()

        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.add([301, 1001])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 100) [200, 300) [301, 1001)')
    
    def test_remove_invalid(self):
        test_range_list = RangeList()
        test_range_list.add([1, 1000])
        
        # Remove invalid
        test_range_list.remove(1)
        test_range_list.remove(['a', 'b'])
        test_range_list.remove([1,1])
        test_range_list.remove([1,2,3])
        
        self.assertEqual(str(test_range_list), '[1, 1000)')
    
    def test_remove_remove_multi_times(self):
        test_range_list = RangeList()
        test_range_list.add([1, 1000])
        
        # Remove invalid
        test_range_list.remove([1, 100])
        self.assertEqual(str(test_range_list), '[100, 1000)')
        test_range_list.remove([100, 200])
        self.assertEqual(str(test_range_list), '[200, 1000)')

        test_range_list.remove([200,400])
        self.assertEqual(str(test_range_list), '[400, 1000)')

        test_range_list.remove([400, 1000])
        self.assertEqual(str(test_range_list), '')

    def test_remove_remove_end_less_than_min(self):
        test_range_list = RangeList()
        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.remove([-10, 0])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 100) [200, 300)')
        
    def test_remove_remove_end_equal_than_min(self):
        test_range_list = RangeList()
        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.remove([-10, 2])
        self.assertEqual(str(test_range_list), '[2, 4) [5, 100) [200, 300)')
        
    def test_remove_remove_full_contained(self):
        test_range_list = RangeList()
        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.remove([200, 400])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 100)')
        
    def test_remove_remove_partial_contained(self):
        test_range_list = RangeList()
        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300]])
        test_range_list.remove([200, 250])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 100) [250, 300)')
        test_range_list.remove([5, 80])
        self.assertEqual(str(test_range_list), '[1, 4) [80, 100) [250, 300)')
        
    def test_remove_remove_cross_multi_range(self):
        test_range_list = RangeList()
        test_range_list._range_list = deque([[1, 4], [5, 100], [200, 300], [300, 1000], [10000, 300000]])
        test_range_list.remove([80, 200000])
        self.assertEqual(str(test_range_list), '[1, 4) [5, 80) [200000, 300000)')
        
  
    def test_with_example(self):
        r1 = RangeList()
        self.assertEqual(str(r1), '')
        r1.add([1, 5])
        self.assertEqual(str(r1), '[1, 5)')
        r1.add([10, 20])
        self.assertEqual(str(r1), '[1, 5) [10, 20)')
        r1.add([20, 20])
        self.assertEqual(str(r1), '[1, 5) [10, 20)')
        r1.add([20, 21])
        self.assertEqual(str(r1), '[1, 5) [10, 21)')
        r1.add([2, 4])
        self.assertEqual(str(r1), '[1, 5) [10, 21)')
        r1.add([3, 8])
        self.assertEqual(str(r1), '[1, 8) [10, 21)')
        r1.remove([10, 10])
        self.assertEqual(str(r1), '[1, 8) [10, 21)')
        r1.remove([10, 11])
        self.assertEqual(str(r1), '[1, 8) [11, 21)')
        r1.remove([15, 17])
        self.assertEqual(str(r1), '[1, 8) [11, 15) [17, 21)')
        r1.remove([3, 19])
        self.assertEqual(str(r1), '[1, 3) [19, 21)')
        

        
suite = unittest.TestLoader().loadTestsFromTestCase(TestRangeList)
unittest.TextTestRunner().run(suite)

..2024-01-12 20:35:22,950 - ERROR - Wrong input with value: None
2024-01-12 20:35:22,951 - ERROR - Wrong input with value: 1
2024-01-12 20:35:22,951 - ERROR - Wrong input with value: ['a', 'b']
2024-01-12 20:35:22,951 - ERROR - Wrong input with value: [1, 1]
2024-01-12 20:35:22,951 - ERROR - Wrong input with value: [1, 2, 3]
......2024-01-12 20:35:22,954 - ERROR - Wrong input with value: 1
2024-01-12 20:35:22,954 - ERROR - Wrong input with value: ['a', 'b']
2024-01-12 20:35:22,955 - ERROR - Wrong input with value: [1, 1]
2024-01-12 20:35:22,955 - ERROR - Wrong input with value: [1, 2, 3]
.......2024-01-12 20:35:22,957 - ERROR - Wrong input with value: [20, 20]
2024-01-12 20:35:22,957 - ERROR - Wrong input with value: [10, 10]
.
----------------------------------------------------------------------
Ran 16 tests in 0.009s

OK


<unittest.runner.TextTestResult run=16 errors=0 failures=0>